### Overview

This notebook contains experiments with grouped bar plots, so we can explore parameterizations for a real API.

**Open questions:**

* Does it make sense to have grouped bars with explicit X coordinates?
    * Since we store the edges of the bars explicitly, they can work fine as the edges of the groups, and are compatible with a `group_width` parameter.
* Does it make sense to have grouped bars with baselines?
    * Grouped bars are incompatible with the BarBoundaries mark by definition, because BarBoundaries explicitly represents $N-1$ series with $N$ boundaries, whereas grouped bars would require $2N$ boundaries to represent $N$ series.  So `baseline=None` cannot be combined with grouped bars, unless we change the definition of the BarBoundaries mark.
        * This is not without its appeal, as a way to merge BarBoundaries and BarMagnitudes?
    * Grouped bars could be represented with a BarMagnitudes mark.  However, the current definition of the baseline would have to change so that there are $MN$ baseline values for $M$ observations and $N$ series.
        * The baselines would have to be stored using multiple columns in the mark data table, or stored outside the table, which seems suboptimal.
        * We'd have to do this whether we allow custom baselines with grouped bars or not, so we might as well allow it.
* How do we specify that bars should be grouped?
    * An extra parameter added to the existing bars() method?
        * This is the best option if we want to allow grouping with custom baselines.
    * A new value for the existing `baseline` parameter?
        * This is the bast option if we don't want to allow grouping with custom baselines.  We might consider renaming `baseline` as `grouping`, `groups`, or something similar.
    * A new API?
        * This doesn't seem desirable at the moment.

In [1]:
import numpy
import toyplot

The following works-around the fact that the Toyplot API doesn't explicitly support grouped bar plots.  We group the bars manually by plotting each series separately while explicitly specifying explicit bar X coordinates to create the grouping.  The `group_width` parameter controls how wide the group is relative to the normal width of a single bar.  This allows us to control the whitespace between groups for clarity.

In [2]:
def grouped_bars(axes, data, group_names, group_width=None):
    if group_width is None:
        group_width = 0.5
    group_left_edges = numpy.arange(data.shape[0], dtype="float") - (group_width / 2.0)
    bar_width = group_width / data.shape[1]
    
    marks = []
    axes.x.ticks.locator = toyplot.locator.Explicit(labels=group_names)
    for index, series in enumerate(data.T):
        left_edges = group_left_edges + (index * bar_width)
        right_edges = group_left_edges + ((index + 1) * bar_width)
        marks.append(axes.bars(left_edges, right_edges, series, opacity=0.5))
        
    return marks

In [3]:
numpy.random.seed(1234)

group_names = [250, 500, 1000, 2000, 3000]

canvas = toyplot.Canvas(width=1000, height=1000)
for index, bar_count in enumerate(numpy.arange(1, 10)):
    axes = canvas.cartesian(grid=(3, 3, index))
    axes.x.label.text = "Number of Runs"
    axes.y.label.text = "Wallclock Time (min)"

    series = numpy.random.uniform(low=0.2, high=1.2, size=(len(group_names), bar_count))
    
    grouped_bars(axes, series, group_names)


250 500 1000 2000 3000 Number of Runs 0.0 0.5 1.0 Wallclock Time (min) 250 500 1000 2000 3000 Number of Runs 0.0 0.4 0.8 1.2 Wallclock Time (min) 250 500 1000 2000 3000 Number of Runs 0.0 0.4 0.8 1.2 Wallclock Time (min) 250 500 1000 2000 3000 Number of Runs 0.0 0.4 0.8 1.2 Wallclock Time (min) 250 500 1000 2000 3000 Number of Runs 0.0 0.4 0.8 1.2 Wallclock Time (min) 250 500 1000 2000 3000 Number of Runs 0.0 0.4 0.8 1.2 Wallclock Time (min) 250 500 1000 2000 3000 Number of Runs 0.0 0.4 0.8 1.2 Wallclock Time (min) 250 500 1000 2000 3000 Number of Runs 0.0 0.4 0.8 1.2 Wallclock Time (min) 250 500 1000 2000 3000 Number of Runs 0.0 0.4 0.8 1.2 Wallclock Time (min) 
 
 
 Save as .csv